In [1]:
import torch
import pandas as pd
import pyaging as pya
import json
import os

In [2]:
%%writefile dunedinpace.r

devtools::install_github("danbelsky/DunedinPACE", build_vignettes = FALSE)

library(DunedinPACE)
library(jsonlite)

PACE_list = list(c(mPACE_Models$model_probes, mPACE_Models$model_weights, mPACE_Models$model_intercept))

write_json(PACE_list, "DunedinPACE.json")

Writing dunedinpace.r


In [3]:
os.system("Rscript dunedinpace.r")
os.system("rm dunedinpace.r")

Error in loadNamespace(x) : there is no package called ‘devtools’
Calls: loadNamespace -> withRestarts -> withOneRestart -> doWithOneRestart
Execution halted


0

In [ ]:
with open('DunedinPACE.json', 'r') as f:
    PACE_list = json.load(f)[0]

In [23]:
features = PACE_list['DunedinPACE']

weights = torch.tensor(PACE_list['DunedinPACE.1']).unsqueeze(0)
intercept = torch.tensor([PACE_list['DunedinPACE.2'][0]])

In [24]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=173, out_features=1, bias=True)
)

In [8]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2022,
    'citation': "Belsky, Daniel W., et al. \"DunedinPACE, a DNA methylation biomarker of the pace of aging.\" Elife 11 (2022): e73420.",
    'doi': "https://doi.org/10.7554/eLife.73420"
}

In [9]:
torch.save(weights_dict, '../weights/dunedinpace.pt')
torch.save(metadata_dict, '../metadata/dunedinpace.pt')

In [10]:
os.system("rm DunedinPACE.json")

rm: DunedinPACE.json: No such file or directory


256